<a href="https://colab.research.google.com/github/NaziaToma/BugType-and-BugFix-Predictor/blob/main/BugFix_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
6from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install nltk scikit-learn

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
# Download necessary NLTK datasets
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Initialize nltk's lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# Define a function to encapsulate preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Lemmatize each word
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

In [ ]:
# Loading datasets
train_df = pd.read_csv('/content/drive/MyDrive/BugFix_TrainingSet.csv')
test_df = pd.read_csv('/content/drive/MyDrive/BugFix_Testset.csv')

In [ ]:
# Fill NaN values with empty strings
train_df.fillna('', inplace=True)
test_df.fillna('', inplace=True)

In [ ]:
# Define the target variable for each dataset
y_train = train_df['Resolution']
y_test = test_df['Resolution']

In [ ]:
# Combine Short Description, bug report, and comments into a single text feature for each dataset
train_df['combined_text'] = train_df['Short Description'] + ' ' + train_df['Bug Report'] + ' ' + train_df[train_df.columns[4:]].apply(lambda x: ' '.join(x.values.tolist()), axis=1)
test_df['combined_text'] = test_df['Short Description'] + ' ' + test_df['Bug Report'] + ' ' + test_df[test_df.columns[4:]].apply(lambda x: ' '.join(x.values.tolist()), axis=1)

In [ ]:
# Applying the preprocessing to each row of the 'combined_text' column
train_df['processed_text'] = train_df['combined_text'].apply(preprocess_text)
test_df['processed_text'] = test_df['combined_text'].apply(preprocess_text)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train = tfidf_vectorizer.fit_transform(train_df['processed_text'])
X_test = tfidf_vectorizer.transform(test_df['processed_text'])

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Train the Random Forest Classifier
rf_classifier = RandomForestClassifier(random_state=80)
rf_classifier.fit(X_train, train_df['Resolution'])

# Predict on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(test_df['Resolution'], y_pred))
print("Classification Report:")
print(classification_report(test_df['Resolution'], y_pred))

Accuracy: 0.7291666666666666
Classification Report:
              precision    recall  f1-score   support

       FIXED       0.75      0.87      0.81        53
     INVALID       0.64      0.80      0.71        46
     WONTFIX       0.88      0.49      0.63        45

    accuracy                           0.73       144
   macro avg       0.76      0.72      0.72       144
weighted avg       0.76      0.73      0.72       144

